You are given an array of CPU tasks, each labeled with a letter from A to Z, and a number n. Each CPU interval can be idle or allow the completion of one task. Tasks can be completed in any order, but there's a constraint: there has to be a gap of at least n intervals between two tasks with the same label.

Return the minimum number of CPU intervals required to complete all tasks.

 

Example 1:

Input: tasks = ["A","A","A","B","B","B"], n = 2

Output: 8

Explanation: A possible sequence is: A -> B -> idle -> A -> B -> idle -> A -> B.

After completing task A, you must wait two intervals before doing A again. The same applies to task B. In the 3rd interval, neither A nor B can be done, so you idle. By the 4th interval, you can do A again as 2 intervals have passed.

Example 2:

Input: tasks = ["A","C","A","B","D","B"], n = 1

Output: 6

Explanation: A possible sequence is: A -> B -> C -> D -> A -> B.

With a cooling interval of 1, you can repeat a task after just one other task.

Example 3:

Input: tasks = ["A","A","A", "B","B","B"], n = 3

Output: 10

Explanation: A possible sequence is: A -> B -> idle -> idle -> A -> B -> idle -> idle -> A -> B.

There are only two types of tasks, A and B, which need to be separated by 3 intervals. This leads to idling twice between repetitions of these tasks.

 

Constraints:

1 <= tasks.length <= 104
tasks[i] is an uppercase English letter.
0 <= n <= 100

# approach 1:
- we need to count the task and keep track of the times we executed them.
- each time unit, we pick the best task to run, the taks wihch is avilable to run and put it in cool down mode.
- if there is no task, then do nothing CPU sits idle.

In [7]:
from collections import Counter

class Solution:
    def leastInterval(self, tasks: list[str], n: int) -> int:
        task_counts = Counter(tasks)
        cooldown = {}
        time = 0
        print(task_counts)

        while task_counts:
            available_tasks = []
            
            # check all the task, and check for their availablity.
            for task in task_counts:
                if task not in cooldown or cooldown[task] < time:
                    available_tasks.append(task)
            print("available tasks", available_tasks)
            if available_tasks:
                # Pick task with max count.
                best_task = max(available_tasks, key=lambda x: task_counts[x])

                # decrease it time.
                task_counts[best_task] -= 1
                if task_counts[best_task] == 0:
                    del task_counts[best_task]
                
                # update the cooldown time for the task.
                cooldown[best_task] = time + n  # next valid time for this task

                print(task_counts)
            # else: idle (nothing to do)

            time += 1
        
        return time

# tc:
# we counter - O(n)
# Worst case: O(T × 26) where T is total time and 26 is max task types to scan each time.
# tc - O(T × 26)
# sc - O(26)


In [8]:
Solution().leastInterval(tasks = ["A","A","A","B","B","B"], n = 2)

Counter({'A': 3, 'B': 3})
available tasks ['A', 'B']
Counter({'B': 3, 'A': 2})
available tasks ['B']
Counter({'A': 2, 'B': 2})
available tasks []
available tasks ['A']
Counter({'B': 2, 'A': 1})
available tasks ['B']
Counter({'A': 1, 'B': 1})
available tasks []
available tasks ['A']
Counter({'B': 1})
available tasks ['B']
Counter()


8

In [9]:
Solution().leastInterval(tasks = ["A","C","A","B","D","B"], n = 1)

Counter({'A': 2, 'B': 2, 'C': 1, 'D': 1})
available tasks ['A', 'C', 'B', 'D']
Counter({'B': 2, 'A': 1, 'C': 1, 'D': 1})
available tasks ['C', 'B', 'D']
Counter({'A': 1, 'C': 1, 'B': 1, 'D': 1})
available tasks ['A', 'C', 'D']
Counter({'C': 1, 'B': 1, 'D': 1})
available tasks ['C', 'B', 'D']
Counter({'B': 1, 'D': 1})
available tasks ['B', 'D']
Counter({'D': 1})
available tasks ['D']
Counter()


6

In [10]:
Solution().leastInterval(tasks = ["A","A","A", "B","B","B"], n = 3)

Counter({'A': 3, 'B': 3})
available tasks ['A', 'B']
Counter({'B': 3, 'A': 2})
available tasks ['B']
Counter({'A': 2, 'B': 2})
available tasks []
available tasks []
available tasks ['A']
Counter({'B': 2, 'A': 1})
available tasks ['B']
Counter({'A': 1, 'B': 1})
available tasks []
available tasks []
available tasks ['A']
Counter({'B': 1})
available tasks ['B']
Counter()


10


---

## ✅ Optimal Approach (Greedy + Math)

### 🧠 Key Idea

We **don’t need to simulate** each time unit. Instead, focus on:

1. Count **how many times the most frequent task appears**.
2. These most frequent tasks will **define the frame of the schedule**.
3. Fill gaps in this frame with other tasks or idle slots.

---

### ✅ Steps

1. **Count the frequency** of all tasks.
2. Find the **maximum frequency (`max_freq`)**.
3. Count how many tasks have this **maximum frequency** (`max_count`).
4. Calculate:

   ```
   Part 1: (max_freq - 1) * (n + 1)
   Part 2: + max_count
   Final Time = max(len(tasks), calculated_time)
   ```

---

### 📌 Formula Explanation

Imagine most frequent task (say 'A') appears 3 times:

```
A _ _ _ A _ _ _ A
```

Each `A` must be spaced by at least `n = 3`. So:

* Number of blocks = max\_freq - 1 = 2
* Size of each block = n + 1 = 4 → total = 2 × 4 = 8
* Add last row's tasks: 8 + 1 = 9

If other tasks can fill the blanks → no idle time
If not enough tasks → idle time is counted.

--

### ✅ Dry Run on: `["A","A","A","B","B","B"], n = 3`

* Frequencies: `A:3, B:3`
* `max_freq = 3`, `max_count = 2`

→ `part1 = (3 - 1) * (3 + 1) = 2 * 4 = 8`
→ `part2 = 2`
→ `total = 8 + 2 = 10`

BUT → `len(tasks) = 6`
So `max(6, 10) = 10`




In [11]:
from collections import Counter

class Solution:
    def leastInterval(self, tasks: list[str], n: int) -> int:
        task_counts = Counter(tasks)
        max_freq = max(task_counts.values())
        max_count = sum(1 for count in task_counts.values() if count == max_freq)

        blocks = (max_freq - 1) 
        size_for_each_blok = (n + 1)

        return max(len(tasks), (blocks * size_for_each_blok) + max_count)
    

# tc `O(n)` → one pass for counting
# sc `O(26)` → counter

In [12]:
Solution().leastInterval(tasks = ["A","A","A", "B","B","B"], n = 3)

10